# Capstone Notebook

This Notebook is created for the Applied Data Science Capstone project

In [1]:
import pandas as pd
import numpy as np


In [4]:
print('Hello Capstone Project Course')

Hello Capstone Project Course


Importing packages needed

In [2]:
import requests
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json


from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

print('Libraries imported.')

Libraries imported.


In [3]:
data_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

## Scaper

In [4]:
data_html = pd.read_html(data_url)

In [5]:
Neighborhoods = data_html[0]
Neighborhoods.head()

Postal Code           Borough                Neighborhood
0         M1A      Not assigned                         NaN
1         M2A      Not assigned                         NaN
2         M3A        North York                   Parkwoods
3         M4A        North York            Victoria Village
4         M5A  Downtown Toronto  Regent Park / Harbourfront

In [6]:
Neighborhoods.dropna(inplace = True, axis = 0)

In [7]:
Neighborhoods.head()

Postal Code           Borough                                  Neighborhood
2         M3A        North York                                     Parkwoods
3         M4A        North York                              Victoria Village
4         M5A  Downtown Toronto                    Regent Park / Harbourfront
5         M6A        North York             Lawrence Manor / Lawrence Heights
6         M7A  Downtown Toronto  Queen's Park / Ontario Provincial Government

In [8]:
Neighborhoods.shape

(103, 3)

# Adding Latitudes and Longitudes

In [9]:
df_ll = pd.read_csv('http://cocl.us/Geospatial_data')
Neighborhoods.keys()

Index(['Postal Code', 'Borough', 'Neighborhood'], dtype='object')

In [10]:
la = []
lo = []
for a in Neighborhoods['Postal Code']:
    temp_index = df_ll[df_ll['Postal Code'].eq(a)]
    temp_la = temp_index['Latitude'].values
    temp_lo = temp_index['Longitude'].values
    la.append(temp_la[0])
    lo.append(temp_lo[0])
Neighborhoods['Latitude'] = la
Neighborhoods['Longitude'] = lo

In [11]:
Neighborhoods.head()

Postal Code           Borough                                  Neighborhood  \
2         M3A        North York                                     Parkwoods   
3         M4A        North York                              Victoria Village   
4         M5A  Downtown Toronto                    Regent Park / Harbourfront   
5         M6A        North York             Lawrence Manor / Lawrence Heights   
6         M7A  Downtown Toronto  Queen's Park / Ontario Provincial Government   

    Latitude  Longitude  
2  43.753259 -79.329656  
3  43.725882 -79.315572  
4  43.654260 -79.360636  
5  43.718518 -79.464763  
6  43.662301 -79.389494

# Creating a map and adding markers for neighborhoods in Toronto

In [12]:
address = ' Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [14]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10.5)

for lat, lng, borough, neighborhood in zip(Neighborhoods['Latitude'], Neighborhoods['Longitude'], Neighborhoods['Borough'], Neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
from IPython.core.display import HTML
m = map_toronto
HTML(m._repr_html_())

## Slicing the data frame to get neighborhoods in Downtown Toronto

In [15]:
downtown_df = Neighborhoods[Neighborhoods['Borough'] == 'Downtown Toronto'].reset_index(drop = True)
downtown_df.head()

Postal Code           Borough                                  Neighborhood  \
0         M5A  Downtown Toronto                    Regent Park / Harbourfront   
1         M7A  Downtown Toronto  Queen's Park / Ontario Provincial Government   
2         M5B  Downtown Toronto                     Garden District / Ryerson   
3         M5C  Downtown Toronto                                St. James Town   
4         M5E  Downtown Toronto                                   Berczy Park   

    Latitude  Longitude  
0  43.654260 -79.360636  
1  43.662301 -79.389494  
2  43.657162 -79.378937  
3  43.651494 -79.375418  
4  43.644771 -79.373306

In [16]:
downtown_address = 'Downtown Toronto, CA'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(downtown_address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


In [17]:
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=13)
for lat, lng, label in zip(downtown_df['Latitude'], downtown_df['Longitude'], downtown_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
m = map_downtown
HTML(m._repr_html_())

## Using Foursquare api to explore

In [18]:
CLIENT_ID = 'SZPQQRN10VAR4YQTKNJLJLWFE5KU40KPSELLPFVV3IQCYTJM'
CLIENT_SECRET = 'QSGXPFTU2KHLWWKVNQ3MS11NWBMJRCMWMMKTQRN0TT5I4CVL'
VERSION = '20180605'

## Getting latitude and longitude of Rosedale to explore using the API

In [19]:
neighborhood_latitude = downtown_df.loc[0, 'Latitude']
neighborhood_longitude = downtown_df.loc[0, 'Longitude'] 
neighborhood_name = downtown_df.loc[0, 'Neighborhood']

## Gettting top 7 places in Rosedale

In [20]:
radius = 1200
LIMIT = 8
api_url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
results = requests.get(api_url).json()

## Extracting the list of venues

In [21]:
venues_temp = results['response']['groups']
venue_dic = venues_temp[0]
venue_t2 = venue_dic['items']
venue_t2 = venue_t2[1:]
venues = []
for v in venue_t2:
    venues.append(v['venue']['name'])

### Top 7 Places to visit in Rosedale

In [22]:
n = 1
for i in venues:
    print('{}. {}'.format(n,i))
    n+=1

1. Tandem Coffee
2. Impact Kitchen
3. The Distillery Historic District
4. Distillery Sunday Market
5. Corktown Common
6. Souk Tabule
7. SOMA chocolatemaker


## Getting 500 venues in Downtown Toronto

In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
LIMIT = 500
downtown_venues = getNearbyVenues(names=downtown_df['Neighborhood'],
                                   latitudes=downtown_df['Latitude'],
                                   longitudes=downtown_df['Longitude']
                                  )

Regent Park / Harbourfront
Queen's Park / Ontario Provincial Government
Garden District / Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond / Adelaide / King
Harbourfront East / Union Station / Toronto Islands
Toronto Dominion Centre / Design Exchange
Commerce Court / Victoria Hotel
University of Toronto / Harbord
Kensington Market / Chinatown / Grange Park
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport
Rosedale
Stn A PO Boxes
St. James Town / Cabbagetown
First Canadian Place / Underground city
Church and Wellesley


In [25]:
downtown_venues.head()

Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0  Regent Park / Harbourfront               43.65426              -79.360636   
1  Regent Park / Harbourfront               43.65426              -79.360636   
2  Regent Park / Harbourfront               43.65426              -79.360636   
3  Regent Park / Harbourfront               43.65426              -79.360636   
4  Regent Park / Harbourfront               43.65426              -79.360636   

                    Venue  Venue Latitude  Venue Longitude  \
0        Roselle Desserts       43.653447       -79.362017   
1           Tandem Coffee       43.653559       -79.361809   
2      Morning Glory Cafe       43.653947       -79.361149   
3  Cooper Koo Family YMCA       43.653249       -79.358008   
4     Body Blitz Spa East       43.654735       -79.359874   

        Venue Category  
0               Bakery  
1          Coffee Shop  
2       Breakfast Spot  
3  Distribution Center  
4                  Spa

In [26]:
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

downtown_onehot['Neighborhood'] = downtown_venues['Neighborhood'] 

fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

## Taking mean frequency of occurance of each category

In [27]:
downtown_grouped = downtown_onehot.groupby('Neighborhood').mean().reset_index()
downtown_grouped

Neighborhood  Yoga Studio  \
0                                         Berczy Park     0.000000   
1   CN Tower / King and Spadina / Railway Lands / ...     0.000000   
2                                  Central Bay Street     0.015625   
3                                            Christie     0.000000   
4                                Church and Wellesley     0.025641   
5                     Commerce Court / Victoria Hotel     0.000000   
6             First Canadian Place / Underground city     0.000000   
7                           Garden District / Ryerson     0.000000   
8   Harbourfront East / Union Station / Toronto Is...     0.000000   
9         Kensington Market / Chinatown / Grange Park     0.000000   
10       Queen's Park / Ontario Provincial Government     0.026316   
11                         Regent Park / Harbourfront     0.020833   
12                         Richmond / Adelaide / King     0.000000   
13                                           Rosedale     0.000000   
14                                     St. James Town     0.000000   
15                       St. James Town / Cabbagetown     0.000000   
16                                     Stn A PO Boxes     0.010526   
17          Toronto Dominion Centre / Design Exchange     0.000000   
18                    University of Toronto / Harbord     0.029412   

    Afghan Restaurant  Airport  Airport Food Court  Airport Gate  \
0            0.000000   0.0000              0.0000        0.0000   
1            0.000000   0.0625              0.0625        0.0625   
2            0.000000   0.0000              0.0000        0.0000   
3            0.000000   0.0000              0.0000        0.0000   
4            0.012821   0.0000              0.0000        0.0000   
5            0.000000   0.0000              0.0000        0.0000   
6            0.000000   0.0000              0.0000        0.0000   
7            0.000000   0.0000              0.0000        0.0000   
8            0.000000   0.0000              0.0000        0.0000   
9            0.000000   0.0000              0.0000        0.0000   
10           0.000000   0.0000              0.0000        0.0000   
11           0.000000   0.0000              0.0000        0.0000   
12           0.000000   0.0000              0.0000        0.0000   
13           0.000000   0.0000              0.0000        0.0000   
14           0.000000   0.0000              0.0000        0.0000   
15           0.000000   0.0000              0.0000        0.0000   
16           0.000000   0.0000              0.0000        0.0000   
17           0.000000   0.0000              0.0000        0.0000   
18           0.000000   0.0000              0.0000        0.0000   

    Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0            0.000            0.000             0.000             0.000000   
1            0.125            0.125             0.125             0.000000   
2            0.000            0.000             0.000             0.000000   
3            0.000            0.000             0.000             0.000000   
4            0.000            0.000             0.000             0.012821   
5            0.000            0.000             0.000             0.040000   
6            0.000            0.000             0.000             0.030000   
7            0.000            0.000             0.000             0.000000   
8            0.000            0.000             0.000             0.000000   
9            0.000            0.000             0.000             0.000000   
10           0.000            0.000             0.000             0.000000   
11           0.000            0.000             0.000             0.000000   
12           0.000            0.000             0.000             0.021277   
13           0.000            0.000             0.000             0.000000   
14           0.000            0.000             0.000             0.038462   
15           0.000            0

In [28]:
num_top_venues = 5

for hood in downtown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2          Restaurant  0.04
3            Beer Bar  0.04
4  Seafood Restaurant  0.04


----CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3     Boat or Ferry  0.06
4          Boutique  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.17
1                Café  0.08
2  Italian Restaurant  0.06
3      Sandwich Place  0.05
4     Bubble Tea Shop  0.03


----Christie----
           venue  freq
0  Grocery Store  0.24
1           Café  0.18
2           Park  0.12
3     Baby Store  0.06
4    Candy Store  0.06


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.09
1     Sushi Restaurant  0.08
2  Japanese Restaurant  0.06
3           Restaurant  0.04
4  

## Sorting venues in ascending order

In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Extracting top 10 venues for each Neighborhood

In [30]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                                        Berczy Park           Coffee Shop   
1  CN Tower / King and Spadina / Railway Lands / ...        Airport Lounge   
2                                 Central Bay Street           Coffee Shop   
3                                           Christie         Grocery Store   
4                               Church and Wellesley           Coffee Shop   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0          Cocktail Bar           Cheese Shop              Beer Bar   
1       Airport Service      Airport Terminal              Boutique   
2                  Café    Italian Restaurant        Sandwich Place   
3                  Café                  Park            Baby Store   
4      Sushi Restaurant   Japanese Restaurant            Restaurant   

  5th Most Common Venue 6th Most Common Venue     7th Most Common Venue  \
0    Seafood Restaurant                Bakery                      Café   
1                 Plane               Airport        Airport Food Court   
2   Japanese Restaurant        Ice Cream Shop                       Bar   
3           Coffee Shop             Nightclub        Athletics & Sports   
4                 Hotel           Men's Store  Mediterranean Restaurant   

  8th Most Common Venue        9th Most Common Venue 10th Most Common Venue  
0            Restaurant  Eastern European Restaurant                  Hotel  
1          Airport Gate                Boat or Ferry                    Bar  
2       Bubble Tea Shop                 Burger Joint           Dessert Shop  
3                 Diner                   Restaurant     Italian Restaurant  
4          Dance Studio                  Yoga Studio                    Pub

### Clustering Neighborhoods depending on basis of most common venues in that location, giving us the idea of what kind of Neighborhood it is

In [31]:
kclusters = 4

downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

kmeans.labels_[0:10] 

array([1, 2, 1, 3, 1, 1, 1, 1, 1, 1])

In [32]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = downtown_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_merged.head() # check the last columns!

Postal Code           Borough                                  Neighborhood  \
0         M5A  Downtown Toronto                    Regent Park / Harbourfront   
1         M7A  Downtown Toronto  Queen's Park / Ontario Provincial Government   
2         M5B  Downtown Toronto                     Garden District / Ryerson   
3         M5C  Downtown Toronto                                St. James Town   
4         M5E  Downtown Toronto                                   Berczy Park   

    Latitude  Longitude  Cluster Labels 1st Most Common Venue  \
0  43.654260 -79.360636               1           Coffee Shop   
1  43.662301 -79.389494               1           Coffee Shop   
2  43.657162 -79.378937               1        Clothing Store   
3  43.651494 -79.375418               1                  Café   
4  43.644771 -79.373306               1           Coffee Shop   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0                Bakery                   Pub                  Park   
1      Sushi Restaurant           Yoga Studio                 Diner   
2           Coffee Shop                  Café            Restaurant   
3           Coffee Shop   American Restaurant             Gastropub   
4          Cocktail Bar           Cheese Shop              Beer Bar   

  5th Most Common Venue      6th Most Common Venue 7th Most Common Venue  \
0            Restaurant                    Theater                  Café   
1              Beer Bar             Sandwich Place   Japanese Restaurant   
2    Italian Restaurant  Middle Eastern Restaurant        Cosmetics Shop   
3          Cocktail Bar             Clothing Store    Italian Restaurant   
4    Seafood Restaurant                     Bakery                  Café   

  8th Most Common Venue        9th Most Common Venue 10th Most Common Venue  
0        Breakfast Spot                   Playground      Electronics Store  
1             Juice Bar          Distribution Center         Discount Store  
2   Japanese Restaurant              Bubble Tea Shop   Fast Food Restaurant  
3    Seafood Restaurant                  Art Gallery               Beer Bar  
4            Restaurant  Eastern European Restaurant                  Hotel

## Visual map reprisentation of clusters

In [33]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighborhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
m = map_clusters
HTML(m._repr_html_())

# Observations

## Cluster 1

In [34]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
14  Downtown Toronto               0                  Park   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
14                 Trail            Playground        Cosmetics Shop   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
14               Dog Run   Distribution Center        Discount Store   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
14                 Diner          Dessert Shop       Department Store

## Cluster 2

In [35]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 1, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
0   Downtown Toronto               1           Coffee Shop   
1   Downtown Toronto               1           Coffee Shop   
2   Downtown Toronto               1        Clothing Store   
3   Downtown Toronto               1                  Café   
4   Downtown Toronto               1           Coffee Shop   
5   Downtown Toronto               1           Coffee Shop   
7   Downtown Toronto               1           Coffee Shop   
8   Downtown Toronto               1           Coffee Shop   
9   Downtown Toronto               1           Coffee Shop   
10  Downtown Toronto               1           Coffee Shop   
11  Downtown Toronto               1                  Café   
12  Downtown Toronto               1                  Café   
15  Downtown Toronto               1           Coffee Shop   
16  Downtown Toronto               1            Restaurant   
17  Downtown Toronto               1           Coffee Shop   
18  Downtown Toronto               1           Coffee Shop   

   2nd Most Common Venue  3rd Most Common Venue 4th Most Common Venue  \
0                 Bakery                    Pub                  Park   
1       Sushi Restaurant            Yoga Studio                 Diner   
2            Coffee Shop                   Café            Restaurant   
3            Coffee Shop    American Restaurant             Gastropub   
4           Cocktail Bar            Cheese Shop              Beer Bar   
5                   Café     Italian Restaurant        Sandwich Place   
7                   Café             Restaurant        Clothing Store   
8               Aquarium                   Café                 Hotel   
9                   Café                  Hotel            Restaurant   
10            Restaurant                   Café                 Hotel   
11            Restaurant                    Bar    Italian Restaurant   
12           Coffee Shop  Vietnamese Restaurant    Mexican Restaurant   
15                  Café               Beer Bar    Seafood Restaurant   
16           Coffee Shop                 Bakery                  Café   
17                  Café             Restaurant                   Gym   
18      Sushi Restaurant    Japanese Restaurant            Restaurant   

   5th Most Common Venue      6th Most Common Venue     7th Most Common Venue  \
0             Restaurant                    Theater                      Café   
1               Beer Bar             Sandwich Place       Japanese Restaurant   
2     Italian Restaurant  Middle Eastern Restaurant            Cosmetics Shop   
3           Cocktail Bar             Clothing Store        Italian Restaurant   
4     Seafood Restaurant                     Bakery                      Café   
5    Japanese Restaurant             Ice Cream Shop                       Bar   
7                  Hotel                        Gym           Thai Restaurant   
8         Scenic Lookout         Italian Restaurant       Fried Chicken Joint   
9    Japanese Restaurant         Italian Restaurant               Salad Place   
10                   Gym        American Restaurant             Deli / Bodega   
11   Japanese Restaurant                  Bookstore                    Bakery   
12                Bakery                Gaming Cafe              Dessert Shop   
15    Italian Restaurant                 Restaurant       Japanese Restaurant   
16                  Park                     Market               Pizza Place   
17   Japanese Restaurant                      Hotel                Steakhouse   
18                 Hotel                Men's Store  Mediterranean Restaurant   

   8th Most Common Venue          9th Most Common Venue 10th Most Common Venue  
0         Breakfast Spot                     Playground      Electronics Store  
1              Juice Bar            Distribution Center         Discount Store  
2    Japanese Restaurant                Bubble Tea Shop   Fast Food Restaurant  
3     Seafood 

## Cluster 3

In [36]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 2, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
13  Downtown Toronto               2        Airport Lounge   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
13       Airport Service      Airport Terminal              Boutique   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
13                 Plane               Airport    Airport Food Court   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
13          Airport Gate         Boat or Ferry                    Bar

## Cluster 4

In [37]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 3, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
6  Downtown Toronto               3         Grocery Store   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
6                  Café                  Park            Baby Store   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
6           Coffee Shop             Nightclub    Athletics & Sports   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
6                 Diner            Restaurant     Italian Restaurant

From the above Clustering result we can see the differnet Neighborhoods, such as 
1. Neighbohoods in Cluster 1 and Cluster 4 are mostly residential neighborhoods with people living there
2. Neighborhoods in Cluster 2 are Commercial Neighborhoods
3. Lastly Cluster 3 represent the Airport area